In [1]:
from iirsBenchmark.regressors import ITEA_regressor, Linear_regressor
from iirsBenchmark.explainers import SHAP_explainer, PermutationImportance_explainer

from sklearn import datasets

housing_data = datasets.fetch_california_housing()
X, y = housing_data['data'], housing_data['target']

linear = Linear_regressor().fit(X, y)

# if you want to specify a parameter, it should be made by named arguments.
# there is few exceptions in iirsBenchmark where arguments are positional.
itea = ITEA_regressor(popsize=50, gens=50, verbose=5).fit(X, y)

print(itea.stochastic_executions)   # True
print(linear.stochastic_executions) # False

print(itea.to_str())   # will print a symbolic equation
print(linear.to_str()) # will print a linear regression equation

gen | smallest fitness | mean fitness | highest fitness | remaining time
----------------------------------------------------------------------------
  0 |         0.765646 |     1.229745 |        1.331550 | 1min8sec    
  5 |         0.799949 |     1.050015 |        1.165380 | 2min3sec    
 10 |         0.770871 |     0.816944 |        0.947580 | 3min4sec    
 15 |         0.706949 |     0.770836 |        0.803728 | 2min45sec   
 20 |         0.676428 |          inf |             inf | 2min6sec    
 25 |         0.610492 |     0.663164 |        0.709189 | 1min45sec   
 30 |         0.609029 |     0.635661 |        0.752983 | 1min13sec   
 35 |         0.585906 |     0.612957 |        0.685914 | 0min58sec   
 40 |         0.556139 |     0.597156 |        0.627084 | 0min38sec   
 45 |         0.545046 |     0.567422 |        0.619896 | 0min19sec   
True
False
0.0*id(x_0^-2 * x_1^-4 * x_2^-4 * x_3^-4 * x_4^2 * x_5 * x_6^3 * x_7^2) + -0.0*sqrt(x_0^4 * x_1 * x_2^-1 * x_3^2 * x_4^2 * x_5^-1

In [2]:
# you must pass the regressor as a named argument for every explainer constructor
shap = SHAP_explainer(predictor=itea).fit(X, y)

# Local explanation takes a matrix where each line is an observation, and
# returns a matrix where each line is the feature importance for the respective input.
# Single observations should be reshaped into a 2D array with x.reshape(1, -1).
local_exps = shap.explain_local(X[5:10, :])
local_exp  = shap.explain_local(X[3].reshape(1, -1))

print(local_exps)
print(local_exp)

[[ 2.42449150e-01  4.06298442e-01  5.14656512e-02  1.33415196e-02
  -2.71422908e-01  2.89319075e-01  3.66932646e-02 -2.79043545e-02]
 [ 5.89693953e-02  4.56378343e-01  9.42245310e-03 -8.07137555e-02
  -3.64994139e-04  3.12595126e-01  3.90928597e-02 -2.51470151e-02]
 [-2.28396693e-01  4.68066097e-01  5.30120261e-02 -9.68271933e-03
   1.44259281e-02  5.19923284e-01  4.11099104e-02 -2.46839677e-02]
 [-7.84592302e-01  2.41349127e-01  1.74814983e-01  1.75959673e-02
   1.76094104e-02  3.61298733e-01  4.19040751e-02 -3.17369923e-02]
 [ 7.80821649e-02  5.29881494e-01 -3.81653447e-03 -6.92249725e-02
   1.35928088e-01  2.98560409e-01  3.85416313e-02 -2.02555806e-02]]
[[ 1.30842294  0.48324006 -0.18422588 -0.00393078 -0.24871136  0.05628855
   0.00347711 -0.02710309]]


In [3]:
# Global explanation take more than one sample (ideally the whole train/test data)
# and returns a single global feature importance for each variable.
pe = PermutationImportance_explainer(predictor=itea).fit(X, y)

global_exp = pe.explain_global(X, y)

print(global_exp)

[[ 2.11713736e+00  1.89619167e-01  2.65937522e-01  3.43862424e-01
   1.30719958e+00  1.33400128e-01 -4.41067571e-03  1.17129807e-03]]


In [ ]:
from iirsBenchmark import expl_measures

# you need to provide a neighborhood to the observation being evaluated
# with those measures

obs_to_explain = X[3].reshape(1, -1)

neighbors = expl_measures.neighborhood(
    obs_to_explain, # The observation 
    X,              # Training data to calculate the multivariate normal distribution
    factor=0.001,   # spread of the neighbors
    size=30         # number of neighbors to sample
)

expl_measures.stability(
    shap,           # the explainer we want to evaluate
    obs_to_explain, # the observation to explain
    neighbors       # sampled neighbors to evaluate the metric
)